Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 4.46 ms, sys: 530 µs, total: 4.99 ms
Wall time: 10.8 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.0.29/8459/1 Dashboard: http://10.20.0.29:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 260 ms, sys: 43.9 ms, total: 304 ms
Wall time: 2.11 s


[1.5957567799127903,
 1.1138728939370979,
 1.3197511743106434,
 1.9386160937919983,
 1.862297132757028,
 1.8676063504385934,
 1.3222295024553676,
 1.139046131734184,
 1.9110127549651812,
 1.7248877836360115,
 1.3583129080686331,
 1.7990775564993462,
 1.9487157612720603,
 1.6630374511882935,
 1.14698917176224,
 1.2825200079784245,
 1.4141967876342358,
 1.147885469254275,
 1.2558973841961545,
 1.6126179953504807,
 1.0887397365910867,
 1.2676392084067678,
 1.4058367140008792,
 1.788643953845072,
 1.6273636727920369,
 1.8818131996496084,
 1.645805844605354,
 1.4788395891598638,
 1.2609243429010515,
 1.511087619990825,
 1.3934641804065193,
 1.9530396316856509,
 1.5094322833289735,
 1.6174719684478172,
 1.4428764432413557,
 1.0090991463438685,
 1.8210948164626468,
 1.0767415995099565,
 1.8145037060230118,
 1.2068398407803347,
 1.8895458255634912,
 1.3705758732961901,
 1.8779178341566483,
 1.7980955350645276,
 1.4118295964374237,
 1.1132846026905496,
 1.5794992816592992,
 1.0533089235482698,


2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.0500433097967368,
 1.0455832815604387,
 1.4525966386308176,
 1.0415933908227721,
 1.0431511144364378,
 1.6649471052067766,
 1.2853039575678369,
 1.5616583297710935,
 1.4983053731697478,
 1.1841103777945472,
 1.0665809073148254,
 1.839731860110517,
 1.5975509426413068,
 1.8641906079088357,
 1.6680157160153701,
 1.8970450911165995,
 1.5341973903073374,
 1.8245975139775337,
 1.3943146231663102,
 1.2108215442421937,
 1.5744240694182703,
 1.4950299654101615,
 1.5109993485525264,
 1.658622272557293,
 1.731027017740678,
 1.3143986424549312,
 1.2316253702067428,
 1.2855287553422237,
 1.0676290921569795,
 1.4580243522765177,
 1.333956776851343,
 1.730541272013288,
 1.8290327599211569,
 1.4584097558697908,
 1.7033382419664598,
 1.749346051970304,
 1.1683538042868056,
 1.2870418096145775,
 1.6395244812195946,
 1.3940875492323643,
 1.4642283906196614,
 1.1155957003451544,
 1.8303885945386997,
 1.3704083817751025,
 1.0229367762326926,
 1.9197453494840242,
 1.6734813430484994,
 1.315941741048213,

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 178 ms, sys: 23.1 ms, total: 201 ms
Wall time: 2.12 s
